In [1]:
from __future__ import print_function

In [2]:
import sys
sys.path.append('../build/')

In [3]:
%pylab inline
np.set_printoptions(precision=4, suppress=True)

Populating the interactive namespace from numpy and matplotlib


In [4]:
import versor as vsr

## Dataset generation

In [478]:
M_object_in_world = vsr.Vec(1.0,1.0,0.1).trs() * vsr.Biv(pi/4.0, 0.0,0.0).exp()
M_eye_in_hand = vsr.Vec(-0.1, -0.2, 0.3).trs() * (vsr.Biv(3.0,-1.0,4.0).unit() * pi/9).exp()
M_eye_in_hand_initial = vsr.Vec(0.11, 0.24, 0.36).trs() * (vsr.Biv(1.0,1.0,0.0).unit() * (np.pi/5.)).exp()

In [479]:
def hand_eye_pose_pairs(M_object_in_world, M_eye_in_hand, n):
    pose_pairs = [
            (M_hand_in_world, 
             (M_eye_in_hand.rev() * M_hand_in_world.rev() * M_object_in_world))
            for M_hand_in_world in [vsr.Vec(*np.random.rand(3)).trs() *
                                    vsr.Rot(vsr.Biv(*np.random.rand(3)).unit() *
                                            np.random.rand() * np.pi)
                                    for i in range(n)]]

    As = [pose_pair[0] for pose_pair in pose_pairs]
    Bs = [pose_pair[1] for pose_pair in pose_pairs]

    LAs = []
    LBs = []
    for i in range(n):
        for j in range(i+1,n):
            LAs.append(((As[j].rev() * As[i]).log() * 0.5).unit())
            LBs.append(((Bs[j] * Bs[i].rev()).log() * 0.5).unit())
    return LAs, LBs

In [480]:
n_lines = 10
lines_a, lines_b = hand_eye_pose_pairs(M_object_in_world, M_eye_in_hand, n_lines)
print(len(lines_a))

45


In [481]:
M = vsr.Vec(1,1,1).trs() * vsr.Biv(pi/12.0, 0.0,0.0).exp()
print(M.matrix())
print(M)

[[ 0.866 -0.5    0.     1.   ]
 [ 0.5    0.866  0.     1.   ]
 [ 0.     0.     1.     1.   ]
 [ 0.     0.     0.     0.   ]]
Mot: [ 0.97 -0.26 0 0 -0.61 -0.35 -0.48 0.13 ]


In [482]:
dual_quat = lambda m : np.array([-m[3], m[2], -m[1], m[0], -m[4], -m[5], -m[6], -m[7] ])
dual_quat(M)

array([-0.    ,  0.    ,  0.2588,  0.9659,  0.6124,  0.3536,  0.483 ,
       -0.1294])

## Hand-Eye Solver

In [491]:
def daniilidis_dualquat(LAs, LBs):
    def skew(v):
        skv = roll(roll(diag(v.flatten()), 1, 1), -1, 0)
        return skv - skv.T
    
    Ds = []
    for LA, LB in zip(LAs, LBs):
        D = np.zeros((6,8))
        a = np.array(LA).copy()[:3]
        a = np.array([a[2], -a[1], a[0]])
        b = np.array(LB).copy()[:3]
        b = np.array([b[2], -b[1], b[0]])
        aprime = np.array(LA).copy()[3:]
        bprime = np.array(LB).copy()[3:]
        # Upper 3
        D[:3,0] = a - b
        D[:3,1:4] = skew(a + b)
        # Lower 3
        D[3:,0] = aprime - bprime
        D[3:,1:4] = skew(aprime + bprime)
        D[3:,4] = a - b 
        D[3:,5:8] = skew(a + b)  
        Ds.append(D)
    Ds = np.array(Ds).reshape(-1,8)
    [U, s, Vt] = np.linalg.svd(Ds)


    v7 = Vt.T[:,-2].copy()
    v8 = Vt.T[:,-1].copy()

    u1 = v7[:4]
    v1 = v7[4:]
    u2 = v8[:4]
    v2 = v8[4:]
    
    a = np.inner(u1,v1)
    b = np.inner(u1,v2) + np.inner(u2,v1)
    c = np.inner(u2,v2)
    [s1, s2] = np.roots([a,b,c])
    
    val1 = (s1**2 * np.inner(u1,u1)) + (2 * s1 * np.inner(u1,u2)) + (np.inner(u2,u2))
    val2 = (s2**2 * np.inner(u1,u1)) + (2 * s2 * np.inner(u1,u2)) + (np.inner(u2,u2))

    if val1 > val2:
        s = s1
        val = val1
    else:
        s = s2
        val = val2

    lambda2 = np.sqrt(1./val)
    lambda1 = s * lambda2
    
#     print(lambda1**2 * a + lambda1 * lambda2 * b + lambda2**2 * c)
#     print((lambda1**2 * np.inner(u1,u1)) + 
#           (2 * lambda1 * lambda2 * np.inner(u1,u2)) + 
#           (lambda2**2 * np.inner(u2,u2)))
    
    
    m_arr = lambda1 * v7 + lambda2 * v8
#     m_arr[1:4] = np.array([m_arr[3], -m_arr[2], m_arr[1]])
#     m_arr[4:8] = np.roll(m_arr[4:8], -1)

    
    return vsr.Mot(*m_arr), lambda1, lambda2, v7, v8

In [502]:
def daniilidis_motor(LAs, LBs):
    
    Ds = []#     print(s1)
#     print(s2)
    for LA, LB in zip(LAs, LBs):
        D = np.zeros((8,8))
        for i in range(8):
            ei = vsr.Mot(0,0,0,0,0,0,0,0)
            ei[i] = 1.0
            D[:,i] = np.array(LA * ei - ei * LB)
        Ds.append(D[1:7,:].copy())
    
    Ds = np.array(Ds).reshape(-1,8)
    [U, s, Vt] = np.linalg.svd(Ds)

    v7 = Vt.T[:,-2].copy()
    v8 = Vt.T[:,-1].copy()
    
    v7 = np.array([v7[0], v7[3], -v7[2], v7[1], -v7[7],v7[4], v7[5], v7[6]])
    v8 = np.array([v8[0], v8[3], -v8[2], v8[1], -v8[7],v8[4], v8[5], v8[6]])
    
    u1 = v7[:4]
    v1 = v7[4:]
    u2 = v8[:4]
    v2 = v8[4:]

    a = np.inner(u1,v1)
    b = np.inner(u1,v2) + np.inner(u2,v1)
    c = np.inner(u2,v2)
    [s1, s2] = np.roots([a,b,c])

    val1 = (s1**2 * np.inner(u1,u1)) + (2 * s1 * np.inner(u1,u2)) + (np.inner(u2,u2))
    val2 = (s2**2 * np.inner(u1,u1)) + (2 * s2 * np.inner(u1,u2)) + (np.inner(u2,u2))
    
    if val1 > val2:
        s = s1
        val = val1
    else:
        s = s2
        val = val2

    lambda2 = np.sqrt(1./val)
    lambda1 = s * lambda2
    
    print(lambda1 * v7 + lambda2 * v8)
    m_arr = lambda1 * Vt.T[:,-2] + lambda2 * Vt.T[:,-1]

    return vsr.Mot(*m_arr), lambda1, lambda2, v7, v8

## Experiments

In [503]:
M_eye_in_hand

Mot: [ 0.94 -0.2 0.067 -0.27 0.077 0.044 -0.16 0.01 ]

In [504]:
m, l11, l21, v71, v81 = daniilidis_motor(lines_a, lines_b)
print(m)
print(np.allclose(M_eye_in_hand, m))

[ 0.9397 -0.2683 -0.0671 -0.2012 -0.0101  0.0772  0.0437 -0.1644]
Mot: [ 0.94 -0.2 0.067 -0.27 0.077 0.044 -0.16 0.01 ]
True


In [495]:
m2, l12, l22, v72, v82 = daniilidis_dualquat(lines_a, lines_b)

print(m2)

Mot: [ -0.94 -0.27 -0.067 -0.2 0.01 0.077 0.044 -0.16 ]


In [488]:
r = np.array(M_eye_in_hand)[:4]
rp = np.array(M_eye_in_hand)[4:]
np.inner(r,r)

1.0

In [406]:
m = M_eye_in_hand
R = vsr.Mot(m[0],m[1],m[2],m[3],0,0,0,0)
Rp = vsr.Mot(0,0,0,0,m[4],m[5],m[6],m[7])

In [408]:
R * R.rev()

Mot: [ 1 0 1.4e-17 0 0 0 0 0 ]

In [393]:
R.rev() * Rp + Rp.rev() * R

Mot: [ 0 0 0 0 0 1.4e-17 0 -3.5e-17 ]

In [394]:
Rp.rev() * R

Mot: [ 0 0 0 0 0.091 0.077 0.14 -1.4e-17 ]

In [472]:
R = np.array([-m[0],m[3],-m[2],m[1]])
Rrev = np.array([-m[0],-m[3],m[2],-m[0]])
Rp = np.array([m[7], m[4],m[5],m[6]])
Rprev = np.array([m[7],-m[4],-m[5],-m[6]])
np.inner(Rrev, Rp) + np.inner(Rprev,R)

-0.1196762840707831

In [399]:
a = np.inner(u1,v1)
b = np.inner(u1,v2) + np.inner(u2,v1)
c = np.inner(u2,v2)
[s1, s2] = np.roots([a,b,c])

Mot: [ 0 0 0 0 -0.091 -0.077 -0.14 -2.1e-17 ]

In [403]:
Rp

Mot: [ 0 0 0 0 -0.066 -0.087 -0.12 0.09 ]

In [404]:
Rp.rev()

Mot: [ 0 -0 -0 -0 0.066 0.087 0.12 0.09 ]

In [384]:
r = vsr.Rot(*np.array(M_eye_in_hand)[:4])
r * r.rev()

Rot: [ 1 0 1.4e-17 0 ]

In [381]:
np.inner(r,np.roll(rp,1))

0.1135375026643278

In [351]:
print(v71)
print(v72)

[-0.8513  0.4446 -0.1482  0.1482  0.0649  0.0852  0.1206 -0.0883]
[ 0.8513  0.1482  0.1482  0.4446 -0.0883  0.0649  0.0852  0.1206]


In [352]:
print(v81)
print(v82)

[ 0.0006 -0.0003  0.0001 -0.0001  0.1507  0.1507  0.4522  0.8661]
[-0.0006 -0.0001 -0.0001 -0.0003  0.8661  0.1507  0.1507  0.4522]


In [353]:
vsr.Mot(*(lambda1 * v7 + lambda2 * v8))

Mot: [ 0.87 -0.45 0.15 -0.15 -0.066 -0.087 -0.12 0.09 ]

In [276]:
M_eye_in_hand

Mot: [ 0.87 -0.45 0.15 -0.15 -0.066 -0.087 -0.12 0.09 ]

In [113]:
a = lines_a[0]
b = lines_b[0]

In [20]:
np.array(vsr.Dll(a[0],0,0,0,0,0) * vsr.Mot(0,1,0,0,0,0,0,0) - 
         vsr.Mot(0,1,0,0,0,0,0,0) * vsr.Dll(b[0],0,0,0,0,0))

array([-0.1968,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ])

In [37]:
l = np.zeros((8,8))
for i in range(8):
    ei = vsr.Mot(0,0,0,0,0,0,0,0)
    ei[i] = 1.0
    l[:,i] += np.array(vsr.Dll(a[0],0,0,0,0,0) * ei - ei * vsr.Dll(b[0],0,0,0,0,0)).copy()
    l[:,i] += np.array(vsr.Dll(0,a[1],0,0,0,0) * ei - ei * vsr.Dll(0,b[1],0,0,0,0)).copy()
    l[:,i] += np.array(vsr.Dll(0,0,a[2],0,0,0) * ei - ei * vsr.Dll(0,0,b[2],0,0,0)).copy()
    l[:,i] += np.array(vsr.Dll(0,0,0,a[3],0,0) * ei - ei * vsr.Dll(0,0,0,b[3],0,0)).copy()
    l[:,i] += np.array(vsr.Dll(0,0,0,0,a[4],0) * ei - ei * vsr.Dll(0,0,0,0,b[4],0)).copy()
    l[:,i] += np.array(vsr.Dll(0,0,0,0,0,a[5]) * ei - ei * vsr.Dll(0,0,0,0,0,b[5])).copy()

print(l)

[[ 0.     -0.1968 -0.1479  0.3448  0.      0.      0.      0.    ]
 [ 0.1968  0.     -0.9971  1.5875  0.      0.      0.      0.    ]
 [ 0.1479  0.9971  0.     -0.5533  0.      0.      0.      0.    ]
 [-0.3448 -1.5875  0.5533  0.      0.      0.      0.      0.    ]
 [ 0.2326  0.7133  0.7166  0.      0.     -0.5533 -1.5875  0.3448]
 [ 0.2099  0.0056  0.      0.7166  0.5533  0.     -0.9971  0.1479]
 [ 0.1153  0.      0.0056 -0.7133  1.5875  0.9971  0.     -0.1968]
 [ 0.      0.1153 -0.2099  0.2326 -0.3448 -0.1479  0.1968  0.    ]]


In [127]:
l = np.zeros((8,8))
for i in range(8):
    ei = vsr.Mot(0,0,0,0,0,0,0,0)
    ei[i] = 1.0
    l[:,i] += np.array(a * ei - ei * b).copy()

print(l[1:7,:])

[[ 0.1968  0.     -0.9971  1.5875  0.      0.      0.      0.    ]
 [ 0.1479  0.9971  0.     -0.5533  0.      0.      0.      0.    ]
 [-0.3448 -1.5875  0.5533  0.      0.      0.      0.      0.    ]
 [ 0.2326  0.7133  0.7166  0.      0.     -0.5533 -1.5875  0.3448]
 [ 0.2099  0.0056  0.      0.7166  0.5533  0.     -0.9971  0.1479]
 [ 0.1153  0.      0.0056 -0.7133  1.5875  0.9971  0.     -0.1968]]


In [115]:
np.array(a * ei - ei * b).copy()

array([ 0.    ,  0.    ,  0.    ,  0.    ,  0.3448,  0.1479, -0.1968,  0.    ])

In [116]:
#         # Upper 3
#         D[:3,0] = a - b
        
#         D[:3,1:4] = np.array([[         0.0,  a[2] + b[2], -a[1] - b[1]],
#                               [-a[2] - b[2],          0.0,  a[0] + b[0]],
#                               [ a[1] + b[1], -a[0] - b[0],          0.0]])
#         # Lower 3
#         D[3:,0] = ap - bp
        
#         D[3:,1:4] = np.array([[-ap[1] - bp[1], -ap[2] - bp[2],            0.0],
#                               [ ap[0] + bp[0],            0.0, -ap[2] - bp[2]],
#                               [           0.0,  ap[0] + bp[0],  ap[1] + bp[1]]])
        
#         D[3:,4:7] = np.array([[         0.0,  a[0] + b[0], a[1] + b[1]],
#                               [-a[0] - b[0],          0.0, a[2] + b[2]],
#                               [-a[1] - b[1], -a[2] - b[2],         0.0]])
        
#         D[3:,7] = np.array([-a[2] + b[2], 
#                              a[1] - b[1], 
#                             -a[0] + b[0]])